#### Imports

In [3]:
import math
import random
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import mne
import itertools

#### Load Data

In [ ]:
# load error dataframe
file_errors_location = 'EinsatzdatenBeatmung.xlsx'
df = pd.read_excel(file_errors_location)
df.head()

In [ ]:
# load vent and ecg data
ventilation_file = "2021-12-13_14-49-27_MMS2_SN14638.edf"
ventilation_data = mne.io.read_raw_edf(ventilation_file)
ventilation_raw_data = ventilation_data.get_data()
ventilation_info = ventilation_data.info
ventilation_channels = ventilation_data.ch_names

ekg_file = "recording.edf"
ekg_data = mne.io.read_raw_edf(ekg_file)
ekg_raw_data = ekg_data.get_data()
# you can get the metadata included in the file and a list of all channels:
ekg_info = ekg_data.info
ekg_channels = ekg_data.ch_names

#### Inspecting the data

In [ ]:
# you can find a list of attributes available here: https://mne.tools/stable/generated/mne.io.Raw.html
ventilation_data.describe(data_frame=True)

In [ ]:
# more information about vent data
ventilation_data

In [ ]:
# more information about ecg data
ekg_data

#### First json format

In [9]:
# first attempt at creating a json for our data
dictionarylist = [
    {
        "name": "PAW",
        "data": ventilation_raw_data[0],
        "resolution": 100
    },
    {
        "name": "EXTFLOWBTPS",
        "data": ventilation_raw_data[1],
        "resolution": 100
    },
    
    {
        "name": "FLOWBTPS",
        "data": ventilation_raw_data[2],
        "resolution": 100
    },
    {
        "name": "CO2_WAVE",
        "data": ventilation_raw_data[3], 
        "resolution": 40
    },
    {
        "name": "EKG_CHANNEL_2",
        "data": ekg_raw_data[0], 
        "resolution": 500
    },
    {
        "name": "EKG_CHANNEL_3",
        "data": ekg_raw_data[1], 
        "resolution": 500
    },
    {
        "name": "NULL",
        "data": ekg_raw_data[2], 
        "resolution": 500
    },
    {
        "name": "PLETH",
        "data": ekg_raw_data[3], 
        "resolution": 500
    },
    {
        "name": "HF",
        "data": ekg_raw_data[4], 
        "resolution": 1
    },
]

#### Storing data between notebooks

In [ ]:
# storing data between notebooks -> used in weinmanndata.ipynb
paw_data = ventilation_raw_data[0]
flow_data = ventilation_raw_data[1]
co2_data = ventilation_raw_data[3]
ecg2_data = ekg_raw_data[0]

%store paw_data
%store ecg2_data
%store flow_data
%store co2_data
%store pleth_data

#### Simple Moving Average

In [10]:
# Calculates the simple moving average
# We use this function for noise reduction in our data
def simple_moving_avg(data, n):
    averaged = []
    for i in range(len(data)):
        temp_value = 0
        for j in range(n):
            # no check for loopback when i-j < 0
            temp_value += data[i-j]
        averaged.append(1/n * temp_value)
    return averaged
     
# Example call for simple_moving_avg
# avg = simple_moving_avg(dictionarylist[5]["data"], 10)

In [11]:
# simple_moving_average for the ekg channels 2 and 3
sma_ekg_2 = simple_moving_avg(dictionarylist[4]["data"], 10)
sma_ekg_3 = simple_moving_avg(dictionarylist[5]["data"], 10)

In [12]:
# generate x-axis
x = [i for i in range(len(sma_ekg_2))]

#### Comparing raw data to the data after using SMA

In [ ]:
# Change is only really visible in a different figsize --> figsize(800, 6) | double click on the whitespace below to see graph
fig, ax = plt.subplots(figsize=(800, 6))
ax.plot(x, sma_ekg_2);

In [ ]:
# No change at all | figsize(800, 6) | double click on the whitespace below to see graph
fig, ax = plt.subplots(figsize=(800, 6))
ax.plot(x, dictionarylist[4]["data"]);

In [ ]:
# writing data into json
with open('data.json', 'w') as f:
    json.dump({"data": dictionarylist}, f)

#### Random Walk

In [ ]:
# generates a list of pseudo random values that are based on the previous value
def make_random_walk(first, last, min_val, max_val, size):
 
    steps = np.random.normal(size=size-2)

   
    #walk = np.cumsum(steps, dtype = int)
    walk = steps.cumsum()
   
    series = walk + first

    # Compare the target min and max values with the observed ones.
    target_min_max = np.array([min_val, max_val])
    observed_min_max = np.array([series.min(), series.max()])

    # Calculate the absolute 'overshoot' for min and max values
    f = np.array([-1, 1])
    overshoot = (observed_min_max*f - target_min_max*f)

    # Calculate the scale factor to constrain the walk within the
    # target min/max values.
    # Don't upscale.
    correction_base = [walk.min(), walk.max()][np.argmax(overshoot)]
    scale = min(1, (correction_base - overshoot.max()) / correction_base)

    # Generate the scaled series
    new_steps = steps * scale
    new_walk = new_steps.cumsum()
    new_series = new_walk + first

    # Check the size of the final step necessary to reach the target endpoint.
    last_step_size = abs(last - new_series[-1]) # step needed to reach desired end

    # Is it larger than the largest previously observed step?
    if last_step_size > np.abs(new_steps).max():
        # If so, consider this series invalid.
        return None
    else:
        # Else, we found a valid series that meets the constraints.
        return np.concatenate((np.array([first]), new_series, np.array([last])))


start = 36
stop = 39
max_val = 43
min_val = 33
size = 817

# Create an infinite stream of candidate series
candidate_walks = (
    (i, make_random_walk(first=start, last=stop, min_val=min_val, max_val=max_val, size=size))
    for i in itertools.count()
)
# Filter out the invalid ones.
valid_walks = ((i, w) for i, w in candidate_walks if w is not None)

idx, walk = next(valid_walks)  # Get the next valid series
print(
    "Walk #{}: min/max({:.2f}/{:.2f})"
    .format(idx, walk.min(), walk.max())
)
plt.plot(walk)
plt.show

#### Generating data with RW

In [160]:
# saving the pseudo random lists
spo2Absolute = walk[0:817]
spo2AbsoluteInt = []
for i in spo2Absolute:
    spo2AbsoluteInt.append(int(i))

co2Absolute = walk
co2AbsoluteInt = []
for i in co2Absolute:
    co2AbsoluteInt.append(int(i))

pleth_data = []
for element in ekg_raw_data[3]:
    pleth_data.append(element)
for element in ekg_raw_data[3][len(ekg_raw_data[3])-153001:-1]:
    pleth_data.append(element)

thin_pleth = []
for k,v in enumerate(pleth_data):
    if ((k % 10) == 1):
        thin_pleth.append(v)

In [165]:
# storing list in between notebooks again
%store spo2AbsoluteInt
%store co2AbsoluteInt

%store thin_hf
%store thin_pleth

#### We tried creating a type of heart attack with the given data

In [173]:
for i in range(75000):
    herzflimmern.append(0)

In [175]:
for k,v in enumerate(sma_ekg_2[90000:105000]):
    herzflimmern.append(v)

In [111]:
import numpy as np

noise = np.random.normal(0,1,45000)

In [126]:
with_noise = 2*np.sin(kammerflimmern) + noise / 8

In [118]:
import random
for k,v in enumerate(noise):
    noise[k] = bool(random.getrandbits(1)) * v

In [161]:
a = -0.000089
b = 67
n = -0.000060
m = 80
l = -0.000006
q = 50
c = 1
d = 0

y, p, dritte, g_curve = [], [], [], []
x = 0
for i in range(45000):
    dritte.append((2*math.pi/q * (x - c)) + d)
    y.append((2*math.pi/b * (x - c)) + d)
    p.append((2*math.pi/m * (x - c)) + d)
    x += 1

In [162]:
f_curve = random.uniform(-0.000093, -0.000089)*np.sin(y)
p_curve = random.uniform(-0.000063, -0.000060)*np.sin(p)
t_curve = random.uniform(-0.000008, -0.000006)*np.sin(dritte)

#for i in range(len(f_curve)):
    #np.append(g_curve, f_curve[i] + p_curve[i] + t_curve[i])
g_curve = f_curve + p_curve + t_curve

In [ ]:

plt.plot(g_curve[0:500])
plt.show

In [ ]:
for k,v in enumerate(g_curve):
    herzflimmern.append(v)
plt.plot(herzflimmern)
plt.show

#### Writing to a json

In [200]:
with open('scenario4_data.json', 'w') as f:
    json.dump({"data": data['data']}, f)